In [1]:
import torch
import torchvision as tv

import pandas as pd
import numpy as np

### 1. Скачайте тренировочную и тестовою часть датасета Fashion MNIST


In [2]:
BatchSize = 256

train_dataset = tv.datasets.MNIST('.', train = True, transform = tv.transforms.ToTensor(), download = True)
test_dataset = tv.datasets.MNIST('.', train=False, transform = tv.transforms.ToTensor(), download = True)
train = torch.utils.data.DataLoader(train_dataset, batch_size = BatchSize, shuffle = True)
test = torch.utils.data.DataLoader(test_dataset, batch_size = BatchSize, shuffle = True)

train_dataset[0][0].shape

torch.Size([1, 28, 28])

### 2. Постройте модель, выбрав стартовую архитектуру


In [3]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [4]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.01)
num_epochs = 10

### 3. Обучите модель и сверьте качество на тестовой части с заданным порогом


In [5]:
import time

def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in train:
            optimizer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            
        print("ep: {}, taked: {:.3f}, train_loss: {:.6f}, train_acc: {:.6f}, test_loss: {:.6f}, test_acc: {:.6f}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [6]:
train_model()

ep: 0, taked: 12.259, train_loss: 0.284967, train_acc: 0.913500, test_loss: 0.114593, test_acc: 0.963600
ep: 1, taked: 12.298, train_loss: 0.165462, train_acc: 0.951950, test_loss: 0.086765, test_acc: 0.972000
ep: 2, taked: 12.462, train_loss: 0.128711, train_acc: 0.961733, test_loss: 0.075969, test_acc: 0.976300
ep: 3, taked: 12.448, train_loss: 0.113221, train_acc: 0.966050, test_loss: 0.074234, test_acc: 0.977000
ep: 4, taked: 12.354, train_loss: 0.096737, train_acc: 0.970933, test_loss: 0.061563, test_acc: 0.980700
ep: 5, taked: 12.398, train_loss: 0.087156, train_acc: 0.973483, test_loss: 0.058681, test_acc: 0.981800
ep: 6, taked: 12.431, train_loss: 0.081981, train_acc: 0.974783, test_loss: 0.058987, test_acc: 0.979800
ep: 7, taked: 12.323, train_loss: 0.072125, train_acc: 0.977900, test_loss: 0.055894, test_acc: 0.982700
ep: 8, taked: 12.395, train_loss: 0.067108, train_acc: 0.978900, test_loss: 0.055818, test_acc: 0.982900
ep: 9, taked: 12.380, train_loss: 0.066480, train_acc: 

### 4. Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. Можно использовать различные оптимизаторы.

In [7]:
# В этом нет смысла, потому что сеть уже обучена на отличный тест.